<a href="https://colab.research.google.com/github/databyhuseyn/DeepLearning/blob/main/Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [4]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

cp: cannot stat 'kaggle.json': No such file or directory


In [5]:
!kaggle datasets download -d ishantjuyal/emotions-in-text

Dataset URL: https://www.kaggle.com/datasets/ishantjuyal/emotions-in-text
License(s): copyright-authors
  0% 0.00/781k [00:00<?, ?B/s]
100% 781k/781k [00:00<00:00, 17.4MB/s]


In [6]:
!wget https://raw.githubusercontent.com/databyhuseyn/DeepLearning/main/helper_functions.py

--2024-05-27 09:59:06--  https://raw.githubusercontent.com/databyhuseyn/DeepLearning/main/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2024-05-27 09:59:06 (66.9 MB/s) - ‘helper_functions.py’ saved [10246/10246]



In [7]:
from helper_functions import unzip_data

In [8]:
unzip_data('/content/emotions-in-text.zip')

In [9]:
data = pd.read_csv('/content/Emotion_final.csv')
data

,Text,Emotion
0,i didnt feel humiliated,sadness
1,i can go from feeling so hopeless to so damned...,sadness
2,im grabbing a minute to post i feel greedy wrong,anger
3,i am ever feeling nostalgic about the fireplac...,love
4,i am feeling grouchy,anger
...,...,...
21454,Melissa stared at her friend in dism,fear
21455,Successive state elections have seen the gover...,fear
21456,Vincent was irritated but not dismay,fear
21457,Kendall-Hume turned back to face the dismayed ...,fear


In [31]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21459 entries, 0 to 21458
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Text     21459 non-null  object
 1   Emotion  21459 non-null  object
dtypes: object(2)
memory usage: 335.4+ KB


In [30]:
data.describe()

,Text,Emotion
count,21459,21459
unique,21405,6
top,i feel so tortured by it,happy
freq,2,7029


In [10]:
df_shuffled = data.sample(frac=1, random_state=42)
df_shuffled

,Text,Emotion
20248,"There was amusement in Jake 's voice , but the...",happy
8094,i feel it is too dangerous to invest in such m...,anger
2859,i feel that i should write the company and tel...,sadness
18953,i also feel ashamed at the hurt caused and ash...,sadness
3931,i just yearned for that homey feeling where yo...,love
...,...,...
11284,i want to savor this feeling of ecstatic antic...,happy
11964,im feeling all puppy dogs and rainbows when im...,sadness
5390,i feel is very delicate,love
860,i was starting to feel a little stressed,sadness


In [11]:
X = df_shuffled['Text']
y = df_shuffled['Emotion']

In [12]:
y.value_counts()

Emotion
happy       7029
sadness     6265
anger       2993
fear        2652
love        1641
surprise     879
Name: count, dtype: int64

In [13]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y)

In [14]:
from sklearn.preprocessing import OneHotEncoder

onehot = OneHotEncoder(sparse=False)

y_train = onehot.fit_transform(y_train.values.reshape(-1, 1))
y_test = onehot.transform(y_test.values.reshape(-1, 1))

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [15]:
X_test.head()

240                                 i feel stressed always
1184     i dont think my desire level is too much to be...
11191    i bought myself a make up palette two months b...
21100                                      I 'm getting ve
7943                          i am feeling fairly virtuous
Name: Text, dtype: object

In [16]:
max_length = round(sum([len(i.split()) for i in X_train]) / len(X_train))

In [17]:
import tensorflow as tf
from tensorflow.keras.layers import TextVectorization

max_vocab_length = 10_000
text_vectorizer = TextVectorization(max_tokens=max_vocab_length,
                                    output_mode='int',
                                    output_sequence_length=max_length)

In [18]:
text_vectorizer.adapt(X_train)

In [19]:
sample_sentence = "He lost his grandma"
text_vectorizer([sample_sentence])

<tf.Tensor: shape=(1, 19), dtype=int64, numpy=
array([[  43,  267,   75, 7270,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0]])>

In [20]:
tf.random.set_seed(42)
from tensorflow.keras import layers
embedding = layers.Embedding(input_dim=max_vocab_length,
                             output_dim=128,
                             embeddings_initializer='uniform',
                             input_length=max_length,
                             name='embedding')

# Model 0:

In [25]:
inputs = layers.Input(shape=(1,), dtype='string')
x = text_vectorizer(inputs)
x = embedding(x)
x = layers.LSTM(64, activation='relu', kernel_regularizer='l2')(x)
outputs = layers.Dense(len(data['Emotion'].value_counts()), activation='sigmoid')(x)
model_0 = tf.keras.Model(inputs, outputs, name='model_0')

In [26]:
model_0.compile(loss='categorical_crossentropy',
                optimizer=tf.keras.optimizers.Adam(),
                metrics='accuracy')

In [27]:
model_0.summary()

Model: "model_0"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization (TextVe  (None, 19)                0         
 ctorization)                                                    
                                                                 
 embedding (Embedding)       (None, 19, 128)           1280000   
                                                                 
 lstm_1 (LSTM)               (None, 64)                49408     
                                                                 
 dense_1 (Dense)             (None, 6)                 390       
                                                                 
Total params: 1329798 (5.07 MB)
Trainable params: 1329798 (5.07 MB)
Non-trainable params: 0 (0.00 Byte)
_____________________

In [28]:
model_0_history = model_0.fit(X_train,
                              y_train,
                              epochs=5,
                              validation_data=(X_test, y_test))

Epoch 1/5
537/537 [==============================] - 24s 37ms/step - loss: 0.9243 - accuracy: 0.7375 - val_loss: 0.8911 - val_accuracy: 0.7784
Epoch 2/5
537/537 [==============================] - 25s 47ms/step - loss: 0.3797 - accuracy: 0.8942 - val_loss: 0.7822 - val_accuracy: 0.7558
Epoch 3/5
537/537 [==============================] - 23s 42ms/step - loss: 0.3159 - accuracy: 0.9048 - val_loss: 0.6856 - val_accuracy: 0.7966
Epoch 4/5
537/537 [==============================] - 18s 34ms/step - loss: 0.2524 - accuracy: 0.9252 - val_loss: 1.1482 - val_accuracy: 0.7905
Epoch 5/5
537/537 [==============================] - 18s 33ms/step - loss: 0.2199 - accuracy: 0.9335 - val_loss: 1.0178 - val_accuracy: 0.7882
